In [1]:
import http.client
import json
import pandas as pd

conn = http.client.HTTPSConnection("web.byui.edu")

payload = ""

headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"}

conn.request("GET", "/studentemployment/api/jobs", payload, headers)

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

In [2]:
# from datetime import datetime
# data_jobs.to_csv(f'../Job_board - Historical Data/data_jobs {datetime.now().strftime("%d-%m-%Y %H%M%S")}.csv')

data_jobs

,jobID,title,departmentName,description,summary,displayJob,dateUpdated,startDate,endDate,managerName,...,beginningDate,recruitingStartDate,requireResume,limitApplicants,limitNumber,applicants,jobQuestions,isOnline,allowOnline,jobMajors
0,1518937,Online Tutor - PSYCH 350,Campus Student,<p><span>Online Tutor is needed for the PSYCH ...,Tutor is needed for the PSYCH 350: Social Psyc...,True,2022-08-17T21:41:06.9460079,2022-08-17T19:00:00,2022-09-11T19:00:00,Casey McDaniel,...,2022-09-12T19:00:00,2020-12-14T01:00:00,True,False,0,[],[],True,False,[]
1,404695,Online Tutor - PSYCH 350,Online Student,<p><span>Online Tutor is needed for the PSYCH ...,Tutor is needed for the PSYCH 350: Social Psyc...,True,2022-08-17T21:40:49.2782597,2022-08-17T09:00:00,2022-09-11T09:00:00,Casey McDaniel,...,2022-09-13T09:00:00,2019-06-25T01:00:00,True,False,0,[],[],False,True,[]
2,341570,Online Tutor - POLSC 150,Campus Student,<p><span> Online Tutor is needed for the POLSC...,Online Tutor is needed for the POLSC 150: Intr...,True,2022-08-17T21:38:56.4515393,2022-08-17T15:00:00,2022-09-08T11:00:00,Casey McDaniel,...,2022-09-13T11:00:00,2018-10-24T01:00:00,True,True,50,[],[],True,False,[]
3,342212,Online Tutor - POLSC 150,Online Student,<p><span>Online Tutor is needed for the POLSC ...,Online Tutor is needed for the POLSC 150. Appl...,True,2022-08-17T21:38:11.0699915,2022-08-17T20:00:00,2022-09-08T16:00:00,Casey McDaniel,...,2022-09-13T16:00:00,2019-05-02T01:00:00,True,True,50,[],[],False,True,[]
4,3647071,Online Tutor - PM 140,Campus Student,<p><b><span>The position requires to be traine...,A tutor is needed for the PM 140 (PMPC 140) cl...,True,2022-08-17T21:36:55.6681526,2022-08-17T19:00:00,2022-09-08T19:00:00,Casey McDaniel,...,2022-09-12T19:00:00,2022-04-08T01:00:00,True,False,0,[],[],True,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,32596,Chemistry Stockroom/TA/Grader,Chemistry,<p>This is a general application form for any ...,General Application form for all chemistry dep...,True,2022-06-09T20:08:58.2027174,2022-06-09T14:00:00,NaN,Mike Wood,...,NaN,2016-11-11T01:00:00,True,False,0,[],[],False,False,"[{'jobMajorId': 212, 'jobId': 32596, 'majorId'..."
396,341293,Security-Lock up Patrol,Public Safety,<p>Employees will assist with patrolling campu...,NaN,True,2022-02-18T23:30:52.6069815,2021-12-05T07:00:00,NaN,Kevin Rhoades,...,NaN,2018-08-15T01:00:00,True,False,0,[],[],False,False,[]
397,1213,Music Production Team (Music Dept.),Music,<p>This position supports productions for the ...,"Audio, video, and stage production support for...",True,2022-02-10T20:45:26.4490587,2022-01-04T22:00:00,NaN,Tyler McNiven,...,2022-01-05T22:00:00,2016-01-20T01:00:00,True,False,0,[],[],False,False,[]
398,341008,Turcotte TA,Physics,<p>Will help set up lab activities and assist ...,Looking for GESCI 208 Robotics TA- 3 positions...,True,2022-01-03T16:37:48.0842611,2022-01-03T19:00:00,NaN,Stephen Turcotte,...,2022-01-03T19:00:00,2018-04-03T01:00:00,True,False,0,[],[],False,False,[]


In [3]:
data_jobs.dateUpdated = data_jobs.dateUpdated.apply(lambda x: pd.to_datetime(x))

data_jobs.startDate = data_jobs.startDate.apply(lambda x: pd.to_datetime(x))

data_jobs.endDate = data_jobs.endDate.apply(lambda x: pd.to_datetime(x))

data_jobs.beginningDate = data_jobs.beginningDate.apply(lambda x: pd.to_datetime(x))

data_jobs.recruitingStartDate = data_jobs.recruitingStartDate.apply(lambda x: pd.to_datetime(x))

columns_to_drop = [
    'jobID', # not needed for EDA
    'description', # not needed for EDA
    'summary', # not needed for EDA
    'displayJob', # single boolean
    'dateUpdated','startDate','endDate', # not needed for display
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]



data_jobs.drop(columns_to_drop, axis=1, inplace=True)


In [4]:
# JOBS FILTERED. NOT ONLINE, NOR CUSTODIAN, AND NOR TA.

remove = ['Online','Custodian','TA']

data_filtered = data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False).reset_index().drop('index',axis=1)

print(f'{data_filtered.shape[0]} JOBS')

pd.set_option('display.max_rows', None)

data_filtered


32 JOBS


,title,departmentName,managerName,payRate,beginningDate,recruitingStartDate,limitApplicants,limitNumber
0,Catering Services Lead,University Food Services,Jesus Del Barrio Sanchez,12.00,2022-09-12 18:00:00,2022-07-28 01:00:00,False,0
1,Health Center Lab Assistant,Student Health Center,Emma Taylor,11.50,2022-09-12 12:00:00,2022-04-20 01:00:00,False,0
2,SRR Curriculum Processor,Student Records & Registration,Laurie Williams,10.70,2022-09-12 12:00:00,2018-10-01 01:00:00,True,40
3,Clinician Aide,Student Health Center,Emma Taylor,10.15,2022-09-01 12:00:00,2022-03-21 01:00:00,False,0
4,Graphic Designer,Music,Maria Nate,10.00,2022-09-02 06:00:00,2018-07-23 01:00:00,False,0
5,Food Services Student Jobs,University Food Services,Fauneil Schultz,10.00,2022-09-14 18:00:00,2022-06-01 01:00:00,False,0
6,Career Success Mentor,Career Center,Sheila Wener,10.00,2022-08-29 14:00:00,2019-11-14 01:00:00,False,0
7,Employer Relations Content Creator,Career & Employment Services,Janet Barton,10.00,2022-09-05 06:00:00,2022-07-26 01:00:00,False,0
8,Student Project Manager,Online Curriculum Development,Brian Carter,10.00,2022-09-09 12:00:00,2016-11-28 01:00:00,True,30
9,Ropes Course Facilitator,Student Activities,Jason Thornton,9.60,2022-09-06 06:00:00,2016-09-01 01:00:00,False,0
